In [ ]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00


In [ ]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

PDF Tracking Tools

In [ ]:
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')

In [ ]:
print("Number of Tagged Sentences ", len(tagged_sentence))
tagged_words = [tup for sent in tagged_sentence for tup in sent]
print("Total Number of Tagged words", len(tagged_words))
vocab = set([word for word,tag in tagged_words])
print("Vocabulary Size", len(vocab))
tags = set([tag for word,tag in tagged_words])
print("Number of Tags in the Corpus ", len(tags))

Number of Tagged Sentences  3914
Total Number of Tagged words 100676
Vocabulary Size 12408
Number of Tags in the Corpus  12


In [ ]:
train_set, test_set = train_test_split(tagged_sentence, test_size=0.2, random_state=1234)
print("Train Set Size", len(train_set))
print("Test Set Size", len(test_set))

Train Set Size 3131
Test Set Size 783


Define the feature function.

สรุปคือเราใช้ CRF Model เพื่อ Train สำหรับ ML สมัยก่อนนั่นเอง

## LSTM POS Tagger

In [ ]:
import nltk
nltk.download('treebank')
tagged_sentences = nltk.corpus.treebank.tagged_sents()

print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words: ", len(nltk.corpus.treebank.tagged_words()) )

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words:  100676


In [ ]:
import numpy as np

sentences, sentence_tags = [], []

for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(sentence)
    sentence_tags.append(tags)

print(sentences[0])
print(sentence_tags[0])

('Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.')
('NNP', 'NNP', ',', 'CD', 'NNS', 'JJ', ',', 'MD', 'VB', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NNP', 'CD', '.')


In [ ]:
from sklearn.model_selection import train_test_split

(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2, random_state=1234)

Neural Network require the number to train.

We need to mapping the word first.

In [ ]:
words, tags = set([]), set([])

for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))} # เริ่มที่ 2 เพราะเผื่อ Padding
word2index['-PAD-'] = 0 # the special value used for padding - ก็คือโปะหน้าหรือหลังด้วยเลข 0
word2index['-OOV-'] = 1 # the speical value used for OOVs - Out of Vocab ตัวไหนไม่อยู่ใน dict ที่ไม่ได้เทรนด์ ให้เป็น 1

tag2index = {t: i + 1 for i, t in enumerate(list(tags))} # เริ่มที่ 1 เพราะเผื่อ O
tag2index['-PAD-'] = 0

In [ ]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

for s in train_sentences:
    s_int = []
    for w in s:
      try:
        s_int.append(word2index[w.lower()])
      except KeyError:
        s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
      try:
        s_int.append(word2index[w.lower()])
      except KeyError:
        s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[2499, 8448, 5290, 5698, 9292, 4395, 8741, 10075, 4316, 1767, 1205, 457, 2657, 6062, 7171, 7306, 5925, 3492, 3614, 2442, 3759, 526, 6450]
[1, 3411, 3387, 1, 1205, 614, 6450]
[41, 39, 39, 35, 20, 35, 19, 41, 42, 14, 37, 15, 25, 23, 9, 12, 10, 30, 19, 41, 35, 15, 32]
[39, 39, 39, 39, 37, 39, 32]


In [ ]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

271


In [ ]:
from keras.preprocessing.sequence import pad_sequences

train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[ 2499  8448  5290  5698  9292  4395  8741 10075  4316  1767  1205   457
  2657  6062  7171  7306  5925  3492  3614  2442  3759   526  6450     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,)))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 271, 128)          1291520   
                                                                 
 bidirectional (Bidirection  (None, 271, 128)          98816     
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 271, 47)           6063      
 ributed)                                                        
                                                                 
 activation (Activation)     (None, 271, 47)           0         
                                                                 
Total params: 1396399 (5.33 MB)
Trainable params: 1396399 (5.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## One-Hot Encoded tags

In [ ]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cat_s = []
        for item in s:
            cat_s.append(np.eye(categories)[item])
        cat_sequences.append(cat_s)
    return np.array(cat_sequences)

In [ ]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

สุดท้ายมันจะบอกว่า คำไหนเป็น Tags ID อะไร มันคือ Tags หรือเป็น Verb หรือ JJ อะไรเป็นต้น มันคือการใช้ LSTM ในการทำ POS tagging สังเกตคือ เราไม่ต้องเตรียม Features เลย โมเดลเรียนรู้เอง